<h1>Collaborative Filtering</h1>

# 0. Matrix Factorization (~SVD)

[1] Matrix factorization techniques for recommender systems.

Note: Not the same with pure SVD in math which decomposite matrix to three sub-matrices.

>  $\min_{q^*, p^*} \sum_{u, i} (r_{u, i} - q_i^T p_u)^2 
+ \lambda(||q_i||^2 + ||p_u||^2)$

## 0.0 SGD

> $e_{u, i} = r_{u, i} - q_i^T p_u$

> $q_i \leftarrow q_i - \eta (- e_{u, i} p_u + \lambda * 2 * q_i )$

> $p_u \leftarrow p_u - \eta (- e_{u, i} q_i + \lambda * 2 * p_u )$

## 0.1 ALS
In each iteration, fix $p_u$ and $q_i$ seperately.

Thus the optimization object becomes covex, 
the problem can be solved using least square methods.

ALS is favorable in at least two cases,

+ Parallelization

+ Implicit Datasets

# 1. Implicit Datasets

[1] Collaborative Filtering for Implicit Feedback Datasets

Let $p_{u, i} = 1$ if $r_{u, i} > 0$ otherwise 0.

> $\min_{x^*, y^*} c_{u, i}(p_{u, i} - x_u^T y_i)^2 
+ \lambda \left(\sum_u||x_u||^2 + \sum_i ||y_i||^2 \right)$

$c_{u, i}$ is the confidence of implicit record, a plausible choice is,

> $c_{u, i} = 1 + \alpha r_{u, i}$

## 1.0 Least Square Methods

> $W^* = min_{W} \frac{1}{2} ||y - X^T W||^2 
= \frac{1}{2} (y - X^T W)^T (y - X^T W)$

To obtain the optimal $W$, compute the gradient, let it be zero, 

> $\frac{\partial{L}}{\partial{W}} = 
\frac{1}{2} \frac{\partial{(y - X^T W)^T (y - X^T W)}}{\partial{W}}$

Recall that in the denominator way,  

> $\vec{u} = \vec{u}(x)$

> $\vec{v} = \vec{v}(x)$

> $\frac{\partial{\vec{u}^T \vec{v}}}{\vec{x}} = 
\frac{\partial{\vec{u}}}{\partial{\vec{x}}} \vec{v} + 
\frac{\partial{\vec{v}}}{\partial{\vec{x}}} \vec{u}$

Let $\vec{u} = y - X^T W$, $\vec{v} = y - X^T W$,

> $\frac{\partial{(y - X^T W)^T (y - X^T W)}}{\partial{W}} = 
\frac{\partial{(y - X^T W)}}{\partial{W}} (y - X^T W) + 
\frac{\partial{(y - X^T W)}}{\partial{W}} (y - X^T W)$

> $ = -2 X (y - X^T W) = 0$

$\Longrightarrow$

> $X y = X X^T W$

If $XX^T$ is inversible, 

> $W^* = (XX^T)^{-1} X y$

## 1.1 Alternative Least Square

Fix $x_u$ and $y_i$ alternativly, 

thus get a convex function of $y_i$ and $x_u$ separatly, 

which can be solved using least square methods. 

# 2. Factorization Machines

[1] Factorization Machines.

## 2.0 2-way factorization machines

> $\hat{y} = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n <v_i, v_j> x_i x_j$

parameters to be learned, 

> $w_0 \in \mathbb{R}$

> $\vec{w} \in \mathbb{R}^n$

> $V \in \mathbb{R} ^ {n \times K}, <v_i, v_j> = \sum_{k=1}^K v_{i, k} v_{j, k}$

<b>Note:</b> If $W$ is a positive definite matrix, there exsits a matrix $V$ such that, 

> $W = V V^T$ 

when $K$ is sufficiently large. In practice, $K$ is offen not so large especially for sparse data.

The original time complexity is $O(Kn^2)$.

Recall that $V$ is <b>symmetric</b> matrix, the pairwise interaction between $x_i$ and $x_j$ ($i < j$) is an upper traingular matrix of $V$ <b>without</b> the diagonal elements.

Hence, 

> $\sum_{i=1}^n \sum_{j=i+1}^n <v_i, v_j> x_i x_j$

> $ = \frac{1}{2} \left \{ 
\sum_{i=1}^n \sum_{j=1}^n <v_i, v_j> x_i x_j  - \sum_{i=1}^n <v_i, v_i> x_i, x_i
\right \}$

> $= \frac{1}{2} \left \{ 
\sum_{i=1}^n \sum_{j=1}^n \sum_{k=1}^K v_{i,k} v_{j,k} x_i x_j - 
\sum_{i=1}^n \sum_{k=1}^{K} v_{i,k} v_{i,k} x_i x_i
\right \}$

> $= \frac{1}{2} \sum_{k=1}^K \left \{ 
\sum_{i=1}^n \sum_{j=1}^n v_{i,k} v_{j,k} x_i x_j - 
\sum_{i=1}^n v_{i,k} v_{i,k} x_i x_i
\right \}$

> $= \frac{1}{2} \sum_{k=1}^K \left \{ 
\left( \sum_{i=1}^n v_{i,k} x_i \right) 
\left( \sum_{j=1}^n v_{j,k} x_j \right) - 
\sum_{i=1}^n (v_{i,k} x_i)^2
\right \}$

> $= \frac{1}{2} \sum_{k=1}^K \left \{ 
\left( \sum_{i=1}^n v_{i,k} x_i \right)^2 - 
\sum_{i=1}^n (v_{i,k} x_i)^2
\right \}$

The time comlexity reduces to $O(K n)$.

## 2.1 D-way Factorization Machines

> $\hat{y} = w_0 + \sum_{i=1}^n w_i x_i +
\sum_{d=2}^D\sum_{i_1=1}^n \sum_{i_2=i_1+1}^n ... \sum_{i_D = i_{D-1}+1}^n
\left( \prod_{j=1}^d x_{i_j} \right)
\left( \sum_{k=1}^K  \prod_{j=1}^d v_{i_j, k}^{(d)}  \right)
$

> $V^{(d)} \in \mathbb{R}^{n \times k_d}$

# 3. Field-aware Factorization Machines 

[1] Field-aware Factorization Machines for CTR Prediction

> $\hat{y} = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n 
<v_{i, f_j}, v_{j, f_i}> x_i x_j$

$f_j$ is the field of $x_j$, $f_i$ is the field of $x_i$.